<a href="https://colab.research.google.com/github/twiese86/Cirq/blob/master/InfoWarfareRussiaNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Information Warfare**
By Thomas L. Wiese III


In [182]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb




import collections

import tweepy as tw
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob

import warnings
warnings.filterwarnings("ignore")

In [183]:
url1 = 'https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_1.csv'
url2 = 'https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_2.csv'
url3 = 'https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_3.csv'
url4 = 'https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_4.csv'
url5 = 'https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_5.csv'
url6 = 'https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_6.csv'
url7 = 'https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_7.csv'
url8 = 'https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_8.csv'
url9 = 'https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_9.csv'
url10 = 'https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_10.csv'
url11 = 'https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_11.csv'
url12 = 'https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_12.csv'
url13 = 'https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_13.csv'

#datasource: https://www.tandfonline.com/doi/abs/10.1080/10584609.2020.1718257
#Darren L. Linvill & Patrick L. Warren (2020) Troll Factories: Manufacturing Specialized Disinformation on Twitter, Political Communication, 37:4, 447-467, DOI: 10.1080/10584609.2020.1718257


In [ ]:
df1 = pd.read_csv(url1,index_col=0,parse_dates=[0]) #read csv data files
df2 = pd.read_csv(url2,index_col=0,parse_dates=[0])
df3 = pd.read_csv(url3,index_col=0,parse_dates=[0])
df4 = pd.read_csv(url4,index_col=0,parse_dates=[0])
df5 = pd.read_csv(url5,index_col=0,parse_dates=[0])
df6 = pd.read_csv(url6,index_col=0,parse_dates=[0])
df7 = pd.read_csv(url7,index_col=0,parse_dates=[0])
df8 = pd.read_csv(url8,index_col=0,parse_dates=[0])
df9 = pd.read_csv(url9,index_col=0,parse_dates=[0])
df10 = pd.read_csv(url10,index_col=0,parse_dates=[0])
df11 = pd.read_csv(url11,index_col=0,parse_dates=[0])
df12 = pd.read_csv(url12,index_col=0,parse_dates=[0])
df13 = pd.read_csv(url13,index_col=0,parse_dates=[0])

In [ ]:
df14 = df1.append(df2)    #append all the datasets into a single dataframe
df15 = df14.append(df3)
df16 = df15.append(df4)
df17 = df16.append(df5)
df18 = df17.append(df6)
df19 = df18.append(df7)
df20 = df19.append(df9)
df21 = df20.append(df10)
df22 = df21.append(df11)
df23 = df22.append(df12)
df24 = df23.append(df13)

In [ ]:
df24.info() #review datatypes and column names


In [ ]:
df24.head() #review a sample fo the data

In [ ]:
df24.isnull().sum()
# this tells me which columns have null values and how many

### **Notes on Null Values from Data Dictionary**


1.   region - A region classification, as determined by Social Studio
2.   post_type	- Indicates if the tweet was a retweet or a quote-tweet
3.   tco#_step1 - tco1_step1	First redirect for the first http(s)://t.co/ link in a tweet, if it exists
tco2_step1	First redirect for the second http(s)://t.co/ link in a tweet, if it exists
tco3_step1	First redirect for the third http(s)://t.co/ link in a tweet, if it exists.





In [ ]:
# Now I'm just going to drop things that are obviously usless but don't nessecarily have null values
df24.drop(['account_type','account_category','article_url'], axis=1, inplace=True)

## Exploratory Data Analysis


In [ ]:
df24

In [ ]:
sb.set_context({"figure.figsize": (50, 25)})
plot = sb.countplot(df24['author'], order=df24.author.value_counts().iloc[:55].index)
plt.xticks(rotation=50)

In [ ]:
import string
string.punctuation


In [ ]:
df24

In [ ]:
df24['language'].value_counts()

In [ ]:
df24[df24["language"].str.contains("English")==True] #drops all non-english tweets

In [ ]:
!pip install tweet-preprocessor
import preprocessor as p
# https://www.kaggle.com/sreejiths0/efficient-tweet-preprocessing

In [ ]:
train_df = df24.dropna()
train_df = df24.drop_duplicates()
train_df.rename(columns = {'content' : 'text'}, inplace = True)

In [ ]:
def preprocess_tweet(row):
    text = row['text']
    text = p.clean(text)
    return text

In [ ]:
train_df['text']=train_df['text'].apply(str)
train_df['text'] = train_df.apply(preprocess_tweet, axis=1) 

#Tweets have been cleaned to normal text.
#Lowercasing
#Punctuation Removal
#Replace extra white spaces
#Stopwords removal
#For stop word removal , i have used gensim library


In [ ]:
train_df.head()


In [ ]:
from gensim.parsing.preprocessing import remove_stopwords


In [ ]:
def stopword_removal(row):
    text = row['text']
    text = remove_stopwords(text)
    return text

In [ ]:
train_df['text'] = train_df.apply(stopword_removal, axis=1) # remove stopwords


In [ ]:
train_df.head()

In [ ]:
train_df['text'] = train_df['text'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')

In [ ]:
train_df.head()


In [ ]:

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
stopwords = set(STOPWORDS)

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=200,
        max_font_size=40, 
        scale=3,
        random_state=1 # chosen at random by flipping a coin; it was heads
    ).generate(str(data))

    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

show_wordcloud(train_df['text'])

In [ ]:
corpus=[]
new= train_df['text'].str.split()
new=new.values.tolist()
corpus=[word for i in new for word in i] #creates a corpus of twittet words

In [ ]:
corpus 

In [ ]:
data_analysis = nltk.FreqDist(corpus)
data_analysis.plot(50, cumulative=False) #seems like there are stop words in here

In [ ]:
train_df.info()

In [ ]:
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))

In [ ]:
filtered_corpus =[]
for w in corpus:
  if w not in stop_words:
    filtered_corpus.append(w)

In [ ]:
data_analysis = nltk.FreqDist(filtered_corpus)
data_analysis.plot(20, cumulative=False) #seems like there are stop words in here

In [ ]:
# this looks better to me, fewer words that seem like stop words
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
res = ' '.join(map(str, filtered_corpus))
wordcloud = WordCloud().generate(res)
#interestingly, the third most common word is police

In [ ]:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer

#people tend to believe what their social network is telling them is true, "friend", and "believe" standout here, mention research into villages and birthcontrol from viral spread of info book

In [ ]:
import collections
def get_top_ngram(corp, n=None):
    vec = CountVectorizer(ngram_range=(n, n)).fit(corp)
    bag_of_words = vec.transform(corp)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) 
                  for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:10]
top_n_bigrams=get_top_ngram(train_df['text'],2)[:10]
x,y=map(list,zip(*top_n_bigrams))
sb.barplot(x=y,y=x)    #bigram

In [ ]:
#trigram
top_tri_grams=get_top_ngram(train_df['text'],n=3)
x,y=map(list,zip(*top_tri_grams))
sb.barplot(x=y,y=x)

In [ ]:
#https://paperswithcode.com/dataset/liar
#starting the ML, reading in and analyzing the LIAR dataset
! pip install fsspec
liarurl= 'https://raw.githubusercontent.com/ekagra-ranjan/fake-news-detection-LIAR-pytorch/master/train.tsv'
df_liar = pd.read_csv(liarurl, sep='\t', error_bad_lines=False, names=["jsonid", "rating", "text", "metatag", 'person', 'personcat','state', 'party', '1', '2', '3', '4', '5', 'source'], index_col=0)

In [ ]:
df_liar.head()
#LIAR: A BENCHMARK DATASET FOR FAKE NEWS DETECTION

#William Yang Wang, "Liar, Liar Pants on Fire": A New Benchmark Dataset for Fake News Detection, to appear in Proceedings of the 55th Annual Meeting of the Association for Computational Linguistics (ACL 2017), short paper, Vancouver, BC, Canada, July 30-August 4, ACL.
#=====================================================================
#Description of the TSV format:

#Column 1: the ID of the statement ([ID].json).
#Column 2: the label.
#Column 3: the statement.
#Column 4: the subject(s).
#Column 5: the speaker.
#Column 6: the speaker's job title.
#Column 7: the state info.
#Column 8: the party affiliation.
#Column 9-13: the total credit history count, including the current statement.
#9: barely true counts.
#10: false counts.
#11: half true counts.
#12: mostly true counts.
#13: pants on fire counts.
#Column 14: the context (venue / location of the speech or statement).

#Note that we do not provide the full-text verdict report in this current version of the dataset,
#but you can use the following command to access the full verdict report and links to the source documents:
#wget http://www.politifact.com//api/v/2/statement/[ID]/?format=json

#======================================================================
#The original sources retain the copyright of the data.

#Note that there are absolutely no guarantees with this data,
#and we provide this dataset "as is",
#but you are welcome to report the issues of the preliminary version
#of this data.

#You are allowed to use this dataset for research purposes only.

#For more question about the dataset, please contact:
#William Wang, william@cs.ucsb.edu

#v1.0 04/23/2017



In [ ]:
# i dont know what columns 1-5 actually represent, need to figure that our
df_rsntwts = train_df

In [ ]:
df_liar.info()

In [ ]:
df_liar['rating'].value_counts()

In [ ]:
df_liar.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
df_liar['ratingnum'] = lb_make.fit_transform(df_liar['rating'])


In [ ]:
df_liar['text'] = df_liar.apply(stopword_removal, axis=1) # remove stopwords
#train_df['text']=train_df['text'].apply(str)
df_liar['text'] = df_liar.apply(preprocess_tweet, axis=1) 
#make corpus of liar text

In [ ]:
df_liar.head()

In [ ]:
import string


In [ ]:
df_liar["text_clean"] = df_liar['text'].str.replace('[^\w\s]','')

In [ ]:
df_liar.head()

In [ ]:
df_liar["text_clean"] = df_liar["text_clean"].str.lower()
df_liar.head()


In [ ]:
from sklearn.model_selection import train_test_split
X = df_liar['text_clean']
y = df_liar['ratingnum']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_final = vectorizer.fit_transform(X_train)

In [ ]:
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                      ('clf', LinearSVC()),
                    ])
text_clf.fit(X_train, y_train)

In [ ]:
predictions = text_clf.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

In [ ]:
# make everything < mostly true = false

In [ ]:
df_liar['ratingnum'].value_counts()

In [ ]:
df_liar['rating'].value_counts()

In [ ]:
df_liar['ratingnum'].mask(df_liar['ratingnum'] == 3, 5, inplace=True) # change mostly-true to true


In [ ]:
df_liar['ratingnum'].mask(df_liar['ratingnum'] == 2, 1, inplace=True)
df_liar['ratingnum'].mask(df_liar['ratingnum'] == 4, 1, inplace=True)
df_liar['ratingnum'].mask(df_liar['ratingnum'] == 0, 1, inplace=True) #change anything <mostly-true to false
# key: 5 = true, 1 = false

In [ ]:
df_liar.head()

In [ ]:
df_liar['ratingnum'].value_counts()

In [ ]:
X = df_liar['text_clean']
y = df_liar['ratingnum']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)

In [ ]:
vectorizer = TfidfVectorizer()
X_train_final = vectorizer.fit_transform(X_train)

In [ ]:
#LinearSVC_classifier = SklearnClassifier(SVC(kernel='linear',probability=True)), LinearSVC()
from nltk.classify.scikitlearn import SklearnClassifier
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                      ('clf', LinearSVC()),
                    ])
text_clf.fit(X_train, y_train)

In [ ]:
predictions = text_clf.predict(X_test)

In [ ]:
print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

In [ ]:
#prediction is 61% accurate for detecting truth vs lie within LIAR dataset
#https://medium.com/@hritikattri10/feature-extraction-using-tf-idf-algorithm-44eedb37305e

In [ ]:
df_rsntwts.head()

In [ ]:
df_rsntwts['ratingnum'] = 1

In [ ]:
df_rsntwts.head()

In [ ]:
X1 = df_rsntwts['text']
Y1 = df_rsntwts['ratingnum']

In [ ]:
predictions = text_clf.predict(X1)

In [ ]:
print(accuracy_score(Y1, predictions))
print(confusion_matrix(Y1, predictions))
print(classification_report(Y1, predictions))#predict if IRA tweets are false using the LIAR dataset assuming all IRA tweets are 'false'

In [ ]:
import tensorflow as tf  #now we are going to use RNN rather than SVC
import re 
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
import seaborn as sns 
plt.style.use('ggplot')

In [ ]:
max_vocab = 10000
tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(X_train) #https://www.kaggle.com/therealcyberlord/fake-news-detection-using-rnn

In [ ]:
# tokenize the text into vectors 
X_train = tokenizer.texts_to_sequences(df_liar['text_clean'] #this is only 80% of the LIAR data, lets try with 100%
X_test = tokenizer.texts_to_sequences(X1)

In [ ]:
y_test = Y1 #similar, lets convert this to ALL LIAR data
y_train = df_liar['ratingnum']

In [ ]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post', maxlen=256)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, padding='post', maxlen=256)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(max_vocab, 32),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

model.summary()

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10,validation_split=0.1, batch_size=30, shuffle=True, callbacks=[early_stop])

In [ ]:
history_dict = history.history

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']
epochs = history.epoch

plt.figure(figsize=(12,9))
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss', size=20)
plt.xlabel('Epochs', size=20)
plt.ylabel('Loss', size=20)
plt.legend(prop={'size': 20})
plt.show()

plt.figure(figsize=(12,9))
plt.plot(epochs, acc, 'g', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy', size=20)
plt.xlabel('Epochs', size=20)
plt.ylabel('Accuracy', size=20)
plt.legend(prop={'size': 20})
plt.ylim((0.5,1))
plt.show()

In [ ]:
#Evaluate the testing set # dont run this

model.evaluate(X_test, y_test)

In [ ]:
pred = model.predict(X_test) #dont run this

binary_predictions = []

for i in pred:
    if i >= 0.5:
        binary_predictions.append(1)
    else:
        binary_predictions.append(0) 

In [ ]:
print('Accuracy on testing set:', accuracy_score(binary_predictions, y_test)) #sont run this
print('Precision on testing set:', precision_score(binary_predictions, y_test))
print('Recall on testing set:', recall_score(binary_predictions, y_test))

In [ ]:
matrix = confusion_matrix(binary_predictions, y_test, normalize='all')#dont run this
plt.figure(figsize=(16, 10))
ax= plt.subplot()
sns.heatmap(matrix, annot=True, ax = ax)

# labels, title and ticks
ax.set_xlabel('Predicted Labels', size=20)
ax.set_ylabel('True Labels', size=20)
ax.set_title('Confusion Matrix', size=20) 
ax.xaxis.set_ticklabels([0,1], size=15)
ax.yaxis.set_ticklabels([0,1], size=15)